# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-widgets
%pip install xgboost==1.5.0
%pip install --upgrade azureml-sdk[notebooks,automl]
%pip install azureml-train-automl-runtime==1.57.0
%pip install azureml-pipeline-core==1.57.0
%pip install azureml-pipeline-steps

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 MB 53.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
  Attempting uninstall: azureml-train-restclients-hyperdrive
    Found existing installation:

In [2]:
import logging
import os
import csv

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.core import Workspace, Experiment, Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)
import json
import requests

SDK version: 1.59.0


In [5]:
import logging
import os
import csv

import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.59.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:


ws = Workspace.from_config(path="Users/odl_user_270675/config.json")
experiment_name = 'automl-project'
project_folder = './my-automl-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-project,quick-starts-ws-270675,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',# for GPU, use "STANDARD_NC6"
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "titanic"
description_text = "titanic dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Maithikimngan/nd00333-capstone/refs/heads/main/train_survival.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# # TODO: Put your automl settings here
# automl_settings = {}

# # TODO: Put your automl config here
# automl_config = AutoMLConfig()

automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 6,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-project,AutoML_a76b2bff-0da2-429e-afa3-2c3f4ca342b2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-project,AutoML_a76b2bff-0da2-429e-afa3-2c3f4ca342b2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number o

{'runId': 'AutoML_a76b2bff-0da2-429e-afa3-2c3f4ca342b2',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-12-16T03:55:20.517702Z',
 'endTimeUtc': '2024-12-16T04:30:38.113593Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'my-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-270675","workspace_name":"quick-starts-ws-270675","region":"westeurope","compute_target":"my-cluster","spark_s

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_automl_run, best_automl_model = remote_run.get_output()
best_automl_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/4b7a3088/classes.interpret.json',
 'explanation/4b7a3088/eval_data_viz.interpret.json',
 'explanation/4b7a3088/expected_values.interpret.json',
 'explanation/4b7a3088/features.interpret.json',
 'explanation/4b7a3088/global_names/0.interpret.json',
 'explanation/4b7a3088/global_rank/0.interpret.json',
 'explanation/4b7a3088/global_values/0.interpret.json',
 'explanation/4b7a3088/local_importance_values.interpret.json',
 'explanation/4b7a3088/per_class_names/0.interpret.json',
 'explanation/4b7a3088/per_class_rank/0.interpret.json',
 'explanation/4b7a3088/per_class_values/0.interpret.json',
 'explanation/4b7a3088/rich_metadata.interpret.json',
 'explanation/4b7a3088/true_ys_viz.interpret.json',
 'explanation/4b7a3088/visualization_dict.interpret.json',
 'explanation/4b7a3088/ys_pred_proba_viz.interpret.json',
 'explanation/4b7a3088/ys_pred_viz.interpret.json',
 'explanation/731ea2e6/classes.interpret.json',
 'expl

In [38]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-project,AutoML_a76b2bff-0da2-429e-afa3-2c3f4ca342b2_40,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
# Save the environement and rename to env.yml
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', "env.yml")

In [12]:
#TODO: Save the best model and rename as best_automl_model
register_best_model = best_automl_run.register_model(model_name="best_automl_model", 
model_path="outputs/model.pkl")

In [39]:
print(best_automl_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('5', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.5944444444444444, learning_rate=0.06316157894736842, max_bin=260, max_depth=6, min_child_weight=2, min_data_in_leaf=0.003457931034482759, min_split_gain=0.42105263157894735, n_estimators=10, n_jobs=1, num_leaves=161, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None, reg_alpha=0.3684210526315789, reg_lambda=0.15789473684210525, subsample=0.5447368421052632))], verbose=False)), ('10', Pipeline(memory=None, steps=[('sparsenormal

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
# Download the score script from auto run and rename to score.py
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', "score.py")

In [15]:
inference_config = InferenceConfig( entry_script='score.py', 
                                    environment=best_automl_run.get_environment())
deploy_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                    memory_gb=1,
                                                    auth_enabled=True,
                                                    enable_app_insights=True)
service=Model.deploy(ws, 
                        "automl-best-model-endpoint", 
                        [register_best_model], 
                        inference_config, 
                        deploy_config)

service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-12-16 06:15:45+00:00 Creating Container Registry if not exists.
2024-12-16 06:15:46+00:00 Registering the environment.
2024-12-16 06:15:47+00:00 Use the existing image.
2024-12-16 06:15:48+00:00 Generating deployment configuration.
2024-12-16 06:15:50+00:00 Submitting deployment to compute..
2024-12-16 06:15:57+00:00 Checking the status of deployment automl-best-model-endpoint..
2024-12-16 06:20:39+00:00 Checking the status of inference endpoint automl-best-model-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [40]:
print(service.state)

Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [28]:
# Show to ground truth data
test = df.head()
ground_truth = test[["Survived"]]
test = test.drop("Survived", axis=1)
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,None,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,None,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,None,S


In [29]:
ground_truth

,Survived
0,False
1,True
2,True
3,True
4,False


TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
request = json.dumps({"data": test.to_dict(orient="records")})
print(request)

{"data": [{"PassengerId": 1, "Pclass": 3, "Name": "Braund, Mr. Owen Harris", "Sex": "male", "Age": 22.0, "SibSp": 1, "Parch": 0, "Ticket": "A/5 21171", "Fare": 7.25, "Cabin": null, "Embarked": "S"}, {"PassengerId": 2, "Pclass": 1, "Name": "Cumings, Mrs. John Bradley (Florence Briggs Thayer)", "Sex": "female", "Age": 38.0, "SibSp": 1, "Parch": 0, "Ticket": "PC 17599", "Fare": 71.2833, "Cabin": "C85", "Embarked": "C"}, {"PassengerId": 3, "Pclass": 3, "Name": "Heikkinen, Miss. Laina", "Sex": "female", "Age": 26.0, "SibSp": 0, "Parch": 0, "Ticket": "STON/O2. 3101282", "Fare": 7.925, "Cabin": null, "Embarked": "S"}, {"PassengerId": 4, "Pclass": 1, "Name": "Futrelle, Mrs. Jacques Heath (Lily May Peel)", "Sex": "female", "Age": 35.0, "SibSp": 1, "Parch": 0, "Ticket": "113803", "Fare": 53.1, "Cabin": "C123", "Embarked": "S"}, {"PassengerId": 5, "Pclass": 3, "Name": "Allen, Mr. William Henry", "Sex": "male", "Age": 35.0, "SibSp": 0, "Parch": 0, "Ticket": "373450", "Fare": 8.05, "Cabin": null, "

In [31]:
key = service.get_keys()[0]
headers = {"Content-type":"application/json"}
headers["Authorization"] = f"Bearer {key}"
res = requests.post(service.scoring_uri, request, headers=headers)

In [32]:
print("Prediction: ", res.text)
print("Ground truth: ", ground_truth.to_dict(orient="records"))

Prediction:  "{\"result\": [false, true, true, true, false]}"
Ground truth:  [{'Survived': False}, {'Survived': True}, {'Survived': True}, {'Survived': True}, {'Survived': False}]


In [33]:
print(service.get_logs())

2024-12-16T06:20:13,342228500+00:00 - rsyslog/run 
2024-12-16T06:20:13,349722100+00:00 - gunicorn/run 
2024-12-16T06:20:13,351973900+00:00 | gunicorn/run | 
2024-12-16T06:20:13,353769000+00:00 | gunicorn/run | ###############################################
2024-12-16T06:20:13,360755700+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-12-16T06:20:13,368714700+00:00 | gunicorn/run | ###############################################
2024-12-16T06:20:13,371918900+00:00 | gunicorn/run | 
2024-12-16T06:20:13,377419200+00:00 | gunicorn/run | 
2024-12-16T06:20:13,381676900+00:00 - nginx/run 
2024-12-16T06:20:13,389959900+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20241202.v2
2024-12-16T06:20:13,396504200+00:00 | gunicorn/run | 
2024-12-16T06:20:13,402686800+00:00 | gunicorn/run | 
2024-12-16T06:20:13,408151800+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
